Notebook that plots raw 12-lead ECG waveformas into a format more familiar to physicians. Update data_directory and file_id. 

Requires biosppy library for now -- but not really using anything critical from that package except for band-pass filter which can be easily subbed.

In [ ]:
from collections import defaultdict
import h5py
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

from biosppy.signals import ecg

# IPython imports
from IPython.display import Image

In [ ]:
#DATA_DIRECTORY = '/Users/pbatra/broad/local_data_sets/tensors/test/2019-03-21'
DATA_DIRECTORY = '/mnt/disks/ecg-rest-31k/2019-06-10/'
leads = {} # dict[name] -> array
file_id = '1000795.hd5'
with h5py.File(os.path.join(DATA_DIRECTORY, file_id)) as hd5:
        for field in hd5['ecg_rest']:
                leads[field] = list(hd5['ecg_rest'][field])

**Biosppy exploration**
* filtering library for a variety of signals, [documentation](https://biosppy.readthedocs.io/en/stable/)
* ecg filtering is meant to be for a single lead only -- does that throw away a lot of useful information across 12 leads?
* also can extract very interesting features like filtered signal, r-preaks, templates, heartbeat templates, heart rate reference, intstantaneous heart rate

**details of filtering**
* looks like the [ecg.ecg](https://github.com/PIA-Group/BioSPPy/blob/212c3dcbdb1ec43b70ba7199deb5eb22bcb78fd0/biosppy/signals/ecg.py) function applies, using [tools.filter_signal](https://github.com/PIA-Group/BioSPPy/blob/212c3dcbdb1ec43b70ba7199deb5eb22bcb78fd0/biosppy/signals/tools.py), by default (most of these are easily modifiable with alternative defaults)  
    * Finite Impulse Response filter ('FIR')
    * bandpass (low-pass, high-pass, band-pass, band-stop all available), 
    * order of the filter is adjustable and set to 0.3 * sampling rate 
    * between 3 and 45 HZ

**Plotting needs:**
* boxes are 1mm x 1mm, 0.04 sec x .1mV 
* horizontal scale is whatever length you want -- (looks like default is 60 boxes)
* vertical scale is 30 boxes, but they can run into each other


**Raw Data**
* lots of fields in raw xml
* extracted is just lead waveform
* for each strip, following is true: sample_rate = 500 Hz, 5000 counts, units (uVperLsb) 5
* for median strips, 600 counts, 500 hz, 5 uVper LSB, first last valid marks
* Vector Loops?
* Full disclosure data? 
* Also have event time stamps

In [ ]:
twelve_leads = defaultdict(dict)
for key, data in leads.items():
    print(key, len(data))
    twelve_leads[key]['raw'] = leads[key]
    if len(data) == 5000:
        (twelve_leads[key]['ts_reference'], twelve_leads[key]['filtered'], twelve_leads[key]['rpeaks'], 
         twelve_leads[key]['template_ts'], twelve_leads[key]['templates'], twelve_leads[key]['heart_rate_ts'], 
         twelve_leads[key]['heart_rate']) = ecg.ecg(signal=leads[key], sampling_rate = 500.) 

In [ ]:
# plot will be in seconds vs mV, boxes are 
sec_per_box = 0.04
mv_per_box = .1
time_interval = 2.5  # time-interval per plot in seconds. ts_Reference data is in s, voltage measurement is 5 uv per lsb
median_interval = 1.2  # 600 samples at 500Hz
raw_scale = 0.005 # convert to mV

fig, ax = plt.subplots(nrows=6, ncols=4, sharey='row', figsize=(24,18), tight_layout=True)
lead_mapping = np.array([['strip_I','strip_aVR', 'strip_V1', 'strip_V4'],
              ['strip_II','strip_aVL', 'strip_V2', 'strip_V5'],
              ['strip_III','strip_aVF', 'strip_V3', 'strip_V6'],
             ])
median_mapping = np.array([['median_I','median_aVR', 'median_V1', 'median_V4'],
              ['median_II','median_aVL', 'median_V2', 'median_V5'],
              ['median_III','median_aVF', 'median_V3', 'median_V6'],
             ])
for i in range(0,3):
    for j in range(0,4):        
        lead_name = median_mapping[i,j]
        lead = twelve_leads[lead_name]
       # ax[i,j].set_xlim(j*time_interval,(j+1)*time_interval)
        ax[i,j].set_ylim(-1.5, 1.5) # -1.5mV -> 1.5mV
        ax[i,j].xaxis.set_major_locator(MultipleLocator(0.2)) # major grids at every .2sec = 5 * 0.04 sec
        ax[i,j].yaxis.set_major_locator(MultipleLocator(0.5)) # major grids at every .5mV 

        ax[i,j].xaxis.set_minor_locator(AutoMinorLocator(5))
        ax[i,j].yaxis.set_minor_locator(AutoMinorLocator(5))

        ax[i,j].grid(which='major', color='#CCCCCC', linestyle='--')
        ax[i,j].grid(which='minor', color='#CCCCCC', linestyle=':')

        ax[i,j].plot(np.arange(0.0, median_interval, median_interval/len(lead['raw'])), [elem_ * raw_scale for elem_ in lead['raw']], label='raw')
        ax[i,j].set_title(lead_name)
        ax[i,j].legend(loc='upper left')
        
for i in range(3,6):
    for j in range(0,4):
        lead_name = lead_mapping[i-3,j]
        lead = twelve_leads[lead_name]
        ax[i,j].set_xlim(j*time_interval,(j+1)*time_interval)
        ax[i,j].set_ylim(-1.5, 1.5) # -1.5mV -> 1.5mV
        ax[i,j].xaxis.set_major_locator(MultipleLocator(0.2)) # major grids at every .2sec = 5 * 0.04 sec
        ax[i,j].yaxis.set_major_locator(MultipleLocator(0.5)) # major grids at every .5mV 

        ax[i,j].xaxis.set_minor_locator(AutoMinorLocator(5))
        ax[i,j].yaxis.set_minor_locator(AutoMinorLocator(5))

        ax[i,j].grid(which='major', color='#CCCCCC', linestyle='--')
        ax[i,j].grid(which='minor', color='#CCCCCC', linestyle=':')
        
        #normalize data in muv
        ax[i,j].plot(lead['ts_reference'], [elem_ * raw_scale for elem_ in lead['filtered']], label='filtered')
        ax[i,j].plot(lead['ts_reference'], [elem_ * raw_scale for elem_ in lead['raw']], label='raw')
        ax[i,j].set_title(lead_name)
        ax[i,j].legend(loc='upper left')
        

fig.savefig(f'{file_id}.pdf')